In [1]:
import pandas as pd
import pymysql
import time
import os

In [2]:
path='data/'

In [4]:
diary_id = 4


In [5]:
def getDF(diary_id):
    diary_map={1:'diary_2',2:'diary_3',3:'diary_8',4:'diary_9'}
    fl = [i for i in os.listdir(path) if i.startswith(diary_map[diary_id])]
#     print len(fl)
    fl = [i for i in fl if os.stat(path+i).st_size/1024/1024.0>1.5]
#     print len(fl)
    
    coll=[]
    for i in fl:
        df0 = pd.read_csv(path+i)
        df0['serial']=int(i.split('_')[-1].split('.csv')[0])
        coll.append(df0)
        del df0
    print len(coll)

    df = pd.concat(coll,axis=0)
    print df.shape
    print 'remove null post_id'
    df = df[(~df['post_ID'].isnull())]
    print df.shape    
    
    seq = sorted(df['serial'].unique())
    date = ['2018-02-21','2018-02-22','2018-02-23','2018-02-24','2018-02-25']
    times = {i:j for i,j in zip(seq,date)}
    df['date']=df['serial'].map(times)
    
    df['post_ID'] = df['post_ID'].astype('int')
    print 'drop duplicates'
    print df.shape
    df = df.drop_duplicates()
    print df.shape
    df = df[['post_ID','date','comments','favor','views']]
    
    return df


    

### post_ID is not null

In [6]:
from secret import dburl, dbusername, dbpasswd
from time import time

db = pymysql.connect(host=dburl,    # your host, usually localhost
                     user=dbusername,         # your username
                     passwd=dbpasswd,  # your password
                     port=3306,
                     db="research",
                     charset='utf8')

cur=db.cursor()

In [7]:
def BatchInsert(df):
    
    df=df[['post_ID','date','comments','favor','views']]
    batch = [df.iloc[n*10000:(n+1)*10000,:] for n in range(0 ,int(df.shape[0]/10000.0)+1)]
    
    start=time()
    n=0
    for d in batch:
        
        sql = """insert ignore into monitorpost values {}""".format(str(map(lambda x: tuple(x),d.values))[1:-1])
        n+=1
        cur.execute(sql)
        db.commit()
        print n,'{:.2f}s |'.format(time()-start),
        start=time()

    

In [8]:
%%time
df = getDF(4)
print df.shape

18
(2542492, 5)
remove null post_id
(560461, 5)
drop duplicates
(560461, 6)
(560461, 6)
(560461, 5)
CPU times: user 916 ms, sys: 204 ms, total: 1.12 s
Wall time: 1.2 s


In [9]:
%%time
BatchInsert(df)

1 1.82s | 2 1.86s | 3 1.88s | 4 1.76s | 5 1.88s | 6 1.85s | 7 1.81s | 8 1.77s | 9 1.73s | 10 1.81s | 11 1.86s | 12 1.76s | 13 1.78s | 14 1.74s | 15 1.76s | 16 1.75s | 17 1.76s | 18 1.73s | 19 1.77s | 20 1.81s | 21 1.98s | 22 1.83s | 23 1.88s | 24 1.88s | 25 1.87s | 26 1.72s | 27 1.80s | 28 1.75s | 29 1.80s | 30 1.79s | 31 1.73s | 32 1.72s | 33 1.82s | 34 1.81s | 35 1.83s | 36 1.79s | 37 1.81s | 38 1.83s | 39 1.86s | 40 1.81s | 41 1.74s | 42 1.73s | 43 1.84s | 44 1.93s | 45 2.39s | 46 2.43s | 47 1.79s | 48 1.79s | 49 1.80s | 50 1.72s | 51 1.73s | 52 1.70s | 53 1.84s | 54 1.78s | 55 1.72s | 56 1.76s | 57 0.13s |CPU times: user 1.79 s, sys: 60 ms, total: 1.85 s
Wall time: 1min 42s

